# 7. Langfuse 기초 실습

## 학습 목표
- Docker로 Langfuse를 셀프 호스팅하기
- Trace logging으로 LLM 호출 추적하기
- LangChain과 통합하여 자동으로 로깅하기
- 대시보드에서 성능 모니터링하기

## 1. Langfuse란?

Langfuse는 LLM 애플리케이션을 위한 **오픈소스 관찰성(observability) 플랫폼**입니다.

### 주요 기능
- 📊 **Tracing**: LLM 호출 과정 추적 및 시각화
- 📈 **Analytics**: 성능 메트릭 및 비용 분석
- 🎯 **Prompt Management**: 프롬프트 버전 관리
- 🔍 **Debugging**: 에러 및 성능 이슈 디버깅

### 왜 필요한가?
- LLM 호출을 가시화
- 응답 시간, 토큰 사용량, 비용 추적
- 프로덕션 환경에서 문제 디버깅

## 2. Docker로 셀프 호스팅

### 가장 쉬운 방법 (추천!)

```bash
# Langfuse 저장소 클론
git clone https://github.com/langfuse/langfuse.git
cd langfuse

# Docker Compose 실행 - 끝!
docker compose up
```

### 실행 확인
```bash
# 백그라운드 실행
docker compose up -d

# 로그 확인
docker compose logs -f

# 중지
docker compose down
```

### 접속
- URL: http://localhost:3000
- 첫 접속 시 회원가입 후 사용

### API 키 발급
1. 대시보드 로그인
2. Settings → API Keys
3. "Create New Key" 클릭
4. Public Key와 Secret Key 복사하여 .env 파일에 저장

```bash
# .env 파일에 추가
LANGFUSE_PUBLIC_KEY=pk-lf-...
LANGFUSE_SECRET_KEY=sk-lf-...
LANGFUSE_HOST=http://localhost:3000
```

## 3. Python SDK 설정

In [ ]:
!pip install langfuse langchain-openai python-dotenv -q

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Langfuse 설정
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY", "pk-lf-...")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY", "sk-lf-...")
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST", "http://localhost:3000")

# OpenAI 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️ .env 파일에 API 키들을 설정하세요")
else:
    print("✅ API 키 로드 완료")

## 4. 기본 Trace Logging

### 간단한 예시

In [ ]:
from langfuse import Langfuse
from langchain_openai import ChatOpenAI

# Langfuse 초기화
langfuse = Langfuse(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

# LLM 호출
llm = ChatOpenAI()
prompt = "Python의 장점 3가지는?"
response = llm.invoke(prompt)

# Trace 생성
trace = langfuse.trace(
    name="simple-llm-call",
    user_id="user-123"
)

# Generation 기록
trace.generation(
    name="openai-call",
    model="gpt-3.5-turbo",
    input=prompt,
    output=response.content,
    usage={
        "prompt_tokens": 10,
        "completion_tokens": 50,
        "total_tokens": 60
    }
)

# 데이터 전송
langfuse.flush()

print("✅ Trace가 Langfuse에 기록되었습니다!")
print(f"대시보드에서 확인하세요: {LANGFUSE_HOST}")

### 복잡한 워크플로우 추적

In [ ]:
import time

def process_query(user_query: str, user_id: str):
    # 메인 trace
    trace = langfuse.trace(
        name="user-query-pipeline",
        user_id=user_id,
        input=user_query,
        metadata={"source": "notebook"}
    )
    
    # Step 1: 입력 검증
    validation_span = trace.span(
        name="validate-input",
        input=user_query
    )
    time.sleep(0.1)
    is_valid = len(user_query) > 0
    validation_span.end(output={"valid": is_valid})
    
    # Step 2: LLM 호출
    llm = ChatOpenAI()
    response = llm.invoke(user_query)
    
    llm_generation = trace.generation(
        name="generate-response",
        model="gpt-3.5-turbo",
        input=user_query,
        output=response.content
    )
    
    # Step 3: 후처리
    postprocess_span = trace.span(
        name="post-process",
        input=response.content
    )
    final_response = response.content.strip()
    postprocess_span.end(output=final_response)
    
    # Trace 종료
    trace.update(output=final_response)
    
    return final_response

# 실행
result = process_query("데코레이터란?", "user-456")
langfuse.flush()

print(f"결과: {result}")
print(f"\n대시보드에서 trace 확인: {LANGFUSE_HOST}")

## 5. LangChain 통합

LangChain 콜백 핸들러를 사용하면 자동으로 추적됩니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langfuse.callback import CallbackHandler

# Langfuse 콜백 핸들러
langfuse_handler = CallbackHandler(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

# LangChain LLM에 콜백 추가
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    callbacks=[langfuse_handler]
)

# 사용 - 자동으로 추적됨!
response = llm.invoke("LangChain이란?")
print(response.content)

print(f"\n✅ 자동으로 Langfuse에 기록되었습니다!")

### Chain에서 사용

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

# 체인 구성
prompt = ChatPromptTemplate.from_template("{topic}에 대해 설명해줘")
llm = ChatOpenAI()
parser = StrOutputParser()

chain = prompt | llm | parser

# 실행 (config에 콜백 전달)
result = chain.invoke(
    {"topic": "FastAPI"},
    config={"callbacks": [langfuse_handler]}
)

print(result)
print(f"\n✅ Chain 실행이 Langfuse에 기록되었습니다!")

### 대화형 챗봇 추적

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langfuse.callback import CallbackHandler

# 설정
langfuse_handler = CallbackHandler(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST,
    user_id="user-789",
    session_id="session-001"
)

llm = ChatOpenAI(callbacks=[langfuse_handler])
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False
)

# 대화 - 모든 턴이 추적됨
print("Bot:", conversation.predict(input="내 이름은 민수야"))
print()
print("Bot:", conversation.predict(input="내 이름이 뭐였지?"))
print()
print("Bot:", conversation.predict(input="Python을 배우고 싶어"))

print(f"\n✅ 전체 대화가 Langfuse에 기록되었습니다!")
print(f"session-001로 대시보드에서 확인하세요.")

## 6. 대시보드 활용

대시보드 (http://localhost:3000)에서 확인할 수 있는 정보:

### Traces 화면
- 모든 trace 목록
- 필터링: 사용자, 날짜, 모델
- 검색: 입력/출력 내용

### Trace 상세 화면
```
Trace: user-query-pipeline
├── Span: validate-input (100ms)
├── Generation: generate-response (500ms)
│   ├── Model: gpt-3.5-turbo
│   ├── Tokens: 60
│   └── Cost: $0.0001
└── Span: post-process (50ms)

Total: 650ms, $0.0001
```

### Analytics 화면
- 시간별 요청 수
- 평균 응답 시간
- 토큰 사용량
- 비용 추이

## 7. 실습 과제

### Level 1
1. Docker로 Langfuse 셀프 호스팅 설정하고 접속하기
2. 간단한 LLM 호출을 trace로 기록하고 대시보드에서 확인
3. LangChain 콜백 핸들러로 자동 추적 구현

In [ ]:
# 과제 1: 기본 trace 생성
# TODO: 코드 작성


### Level 2
4. 여러 span을 포함한 복잡한 워크플로우 추적하기
5. 사용자별로 다른 session_id를 부여하여 세션 관리
6. 에러 발생 시 trace.event()로 기록하기

In [ ]:
# 과제 2: 복잡한 워크플로우
# TODO: 코드 작성


### Level 3
7. ConversationChain + Langfuse로 전체 대화 세션 추적
8. 대시보드 Analytics에서 성능 병목 구간 찾기
9. 프롬프트 버전 관리 및 A/B 테스트 (대시보드 사용)

In [ ]:
# 과제 3: 세션 추적
# TODO: 코드 작성


## 정리

Langfuse의 장점:
- ✅ LLM 호출 과정을 완전히 가시화
- ✅ 응답 시간, 토큰, 비용 자동 추적
- ✅ LangChain과 네이티브 통합
- ✅ 프로덕션 환경에서 디버깅 용이
- ✅ 셀프 호스팅으로 데이터 프라이버시 보장

## 다음 단계

LangChain + Langfuse 통합 예시를 확인하세요!

👉 [langchain_langfuse_example.py](langchain_langfuse_example.py)